# Analysis of the autism data

In [122]:
run init.ipynb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Done.


# Experiments

* This section aims at answering a few questions related to the different algorithms used, with the aim of optimizing our classification framework.

**Parameters or potential settings**

* The approach used, which could be Logistic Regression, Decision Tree, Explainable Boosting Machine, Neural Additive models, or XGBoost.
* The use or not of indicator variables as inputs. 
* The scaling of the data. 
* The imputation approach when the learning algorithms does not handle them by design. It could be constant imputation (called encoding here), or conditional imputation (mean, knn, or mice).
* The sampling method in the case of imbalance learning (either no down-sampling, vanilla (random sampling of the minority class until classes are balanced), or SMOTE (a more elaborated sampling).
* The number of features, between 2 and 6. 
* The number of folds when cross-validating the results. 

**Notes on the classification pipeline:**

* Two datasets are age-matched but one (scenario `asd_td_age_matched_n_balanced`) lead to relatively balanced classes, whereas the scenario `asd_td_age_matched_n_unbalanced` has slightly younger kids and so leverage the amount of young neuro-typical kids, and is more unbalanced. 
* No searches on hyper-parameters are performed on each of the approach. Cross-validation is performed with the stratified inner-fold being left-out, the model is fitted on the training set, and the prediction of the test set is done with predictions stored for later evaluation of performances. 
* since a lot of settings parameters are tested, we test all of the hypothesis with a default setting that is: Encoding of the missing variables, scaling of the data, no use of indicator variables, no down-sampling (???), and a 16-fold cross-validation.
* Features selection were made for the two scenario suing the features with the higher importance based on xgboost importance map.
* Classification here is between autistic and neurotypical participants.



**Among the questions at stakes:**

* Experiment 1: Shall we use indicator variables? For each scenario (columns of axes), ad each dimension of the problem (rows of axes) , x is the `use_of_indicator_variables`, y is an indicator of performances (typically the F1 score), and the hue variable are the approaches. The plots are performed for no imputations, 
* Experiement 2: Shall we scale data or not ? 
* Experiment X: for the algorithms handling missing variables (xgboost, nam with encoding) , shall we let them be missing, or imputed ? 

In [4]:
df =  pd.read_csv(AUTISM_DATA_PATH)

scenario='papers'

with open(os.path.join(DATA_DIR, 'selected_features_{}.pkl'.format(scenario)), 'rb') as f:
    features = pickle.load(f)

## Experiment 1.  `use_missing_indicator_variables`

In [45]:
n=0
for i, n_features in enumerate(list(features['xgboost'].keys())):
    for j, scenario in enumerate(['papers']):
        
        for use_missing_indicator_variables in [True, False]:
            
            for approach in ["LogisticRegression", "DecisionTree",  "ebm", 'xgboost', "nam"]:
                
                df =  pd.read_csv(AUTISM_DATA_PATH)


                data = Dataset(df=df, 
                               missing_data_handling='encoding',
                               imputation_method='without',
                               sampling_method='without',
                               scenario = scenario, 
                               features_name = features[approach][n_features] if approach in features.keys() else features['ebm'][n_features],
                               scale_data=True, 
                               use_missing_indicator_variables=use_missing_indicator_variables,
                               verbosity=1, 
                               proportion_train=1)

                exp = Experiments(data.dataset_name,
                                  dataset=data, 
                                  approach=approach, 
                                  previous_experiment=None,        
                                  debug=True, 
                                  verbosity=0, 
                                  experiment_folder_name='paper_experiment_1_fs',
                                  save_experiment=True)
                print("approach: {} - use_missing_indicator_variables: {} - n_features: {}".format(approach, use_missing_indicator_variables, n_features))
                exp.fit_predict(num_cv=16)
                display(exp.performances_df)
                

approach: LogisticRegression - use_missing_indicator_variables: True - n_features: 10


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.729,0.264,0.81,0.083,0.845,0.356,0.18,0.832,0.266,0.465,0.87,0.27,0.94,0.535,0.73,0.06


approach: DecisionTree - use_missing_indicator_variables: True - n_features: 10


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.659,0.372,0.878,0.202,0.964,0.406,0.319,0.915,0.359,0.256,0.983,0.611,0.927,0.744,0.389,0.073


approach: ebm - use_missing_indicator_variables: True - n_features: 10


...

,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.859,0.415,0.906,0.159,0.929,0.475,0.259,0.863,0.408,0.628,0.887,0.365,0.958,0.372,0.635,0.042


approach: xgboost - use_missing_indicator_variables: False - n_features: 32


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.821,0.443,0.92,0.232,0.979,0.458,0.332,0.885,0.38,0.488,0.925,0.404,0.946,0.512,0.596,0.054


approach: nam - use_missing_indicator_variables: False - n_features: 32


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV),Area Under the Curve (AUC)
0,0.8392,0.3837,0.8778,0.1606,0.9428,0.4248,0.2268,0.8636,0.3476,0.507,0.9004,0.3532,0.9468,0.493,0.6468,0.0532,0.8392


In [73]:
n=0
use_missing_indicator_variables = {'FunAtThePark':['FP_postural_sway', 'FP_postural_sway_derivative'],
                                           'RhymesAndToys': ['RT_postural_sway', 'RT_postural_sway_derivative']}

for i, n_features in enumerate(list(features['xgboost'].keys())):
    for j, scenario in enumerate(['papers']):

        for approach in ["LogisticRegression", "DecisionTree",  "ebm", 'xgboost', "nam"]:

            df =  pd.read_csv(AUTISM_DATA_PATH)


            data = Dataset(df=df, 
                           missing_data_handling='encoding',
                           imputation_method='without',
                           sampling_method='without',
                           scenario = scenario, 
                           features_name = features[approach][n_features] if approach in features.keys() else features['ebm'][n_features],
                           scale_data=True, 
                           use_missing_indicator_variables=use_missing_indicator_variables,
                           verbosity=1, 
                           proportion_train=1)

            exp = Experiments(data.dataset_name,
                              dataset=data, 
                              approach=approach, 
                              previous_experiment=None,        
                              debug=True, 
                              verbosity=0, 
                              experiment_folder_name='paper_experiment_1_fs_custom_missing',
                              save_experiment=True)
            print("approach: {} - use_missing_indicator_variables: {} - n_features: {}".format(approach, use_missing_indicator_variables, n_features))
            exp.fit_predict(num_cv=16)
            display(exp.performances_df)


approach: LogisticRegression - use_missing_indicator_variables: {'FunAtThePark': ['FP_postural_sway', 'FP_postural_sway_derivative'], 'RhymesAndToys': ['RT_postural_sway', 'RT_postural_sway_derivative']} - n_features: 10


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.745,0.268,0.804,0.082,0.817,0.381,0.196,0.856,0.293,0.442,0.899,0.311,0.94,0.558,0.689,0.06


approach: DecisionTree - use_missing_indicator_variables: {'FunAtThePark': ['FP_postural_sway', 'FP_postural_sway_derivative'], 'RhymesAndToys': ['RT_postural_sway', 'RT_postural_sway_derivative']} - n_features: 10


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.735,0.474,0.931,0.331,0.984,0.48,0.407,0.915,0.418,0.372,0.971,0.571,0.937,0.628,0.429,0.063


approach: ebm - use_missing_indicator_variables: {'FunAtThePark': ['FP_postural_sway', 'FP_postural_sway_derivative'], 'RhymesAndToys': ['RT_postural_sway', 'RT_postural_sway_derivative']} - n_features: 10


...

,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.861,0.417,0.895,0.198,0.974,0.451,0.238,0.876,0.373,0.512,0.913,0.379,0.948,0.488,0.621,0.052


approach: xgboost - use_missing_indicator_variables: {'FunAtThePark': ['FP_postural_sway', 'FP_postural_sway_derivative'], 'RhymesAndToys': ['RT_postural_sway', 'RT_postural_sway_derivative']} - n_features: 32


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.852,0.472,0.936,0.266,0.986,0.46,0.306,0.88,0.383,0.512,0.918,0.393,0.948,0.488,0.607,0.052


approach: nam - use_missing_indicator_variables: {'FunAtThePark': ['FP_postural_sway', 'FP_postural_sway_derivative'], 'RhymesAndToys': ['RT_postural_sway', 'RT_postural_sway_derivative']} - n_features: 32


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV),Area Under the Curve (AUC)
0,0.8364,0.3657,0.8712,0.1432,0.9122,0.4087,0.2351,0.8416,0.3344,0.5534,0.8715,0.3161,0.9501,0.4466,0.6839,0.0499,0.8364


In [118]:
df =  pd.read_csv(AUTISM_DATA_PATH)

scenario='papers'

with open(os.path.join(DATA_DIR, 'selected_features_{}.pkl'.format(scenario)), 'rb') as f:
    features = pickle.load(f)
    
n=0
use_missing_indicator_variables = {'Floating Bubbles': ['FB_postural_sway', 'FB_postural_sway_derivative'],
                                 'DogInGrassC': ['DIGC_postural_sway', 'DIGC_postural_sway_derivative'],
                                 'DogInGrassRRL': ['DIGRRL_postural_sway', 'DIGRRL_postural_sway_derivative'],
                                 'SpinningTop': ['ST_postural_sway', 'ST_postural_sway_derivative'],
                                 'Social gaze': ['S_gaze_percent_right', 'S_gaze_silhouette_score'],
                                 'PlayingWithBlocks': ['PWB_postural_sway', 'PWB_postural_sway_derivative'],
                                 'FunAtThePark': ['FP_postural_sway', 'FP_postural_sway_derivative'],
                                 'MechanicalPuppy': ['MP_postural_sway', 'MP_postural_sway_derivative'],
                                 'BlowingBubbles': ['BB_postural_sway', 'BB_postural_sway_derivative'],
                                 'RhymesAndToys': ['RT_postural_sway', 'RT_postural_sway_derivative'],
                                 'MakeMeLaugh': ['MML_postural_sway', 'MML_postural_sway_derivative'],
                                 'Game administration': ['number_of_touches', 'number_of_target', 'exploratory_percentage'],
                                 'No touches at all': ['average_length',
                                  'std_length',
                                  'pop_rate',
                                  'average_touch_duration',
                                  'std_touch_duration',
                                  'average_accuracy_variation',
                                  'accuracy_consistency'],
                                 'Game grouped touches': ['average_time_spent', 'std_time_spent']}


for i, n_features in enumerate(list(features['xgboost'].keys())):
    for j, scenario in enumerate(['papers']):
        for approach in ["ebm"]:
            for feature_missing, feats in use_missing_indicator_variables.items():

                df =  pd.read_csv(AUTISM_DATA_PATH)

                data = Dataset(df=df, 
                               missing_data_handling='encoding',
                               imputation_method='without',
                               sampling_method='without',
                               scenario = scenario, 
                               features_name = features[approach][n_features] if approach in features.keys() else features['ebm'][n_features],
                               scale_data=True, 
                               use_missing_indicator_variables={feature_missing:feats},
                               verbosity=1, 
                               proportion_train=1)

                exp = Experiments(data.dataset_name,
                                  dataset=data, 
                                  approach=approach, 
                                  previous_experiment=None,        
                                  debug=True, 
                                  verbosity=0, 
                                  experiment_folder_name='paper_experiment_1_fs_custom_missing_1_per_1',
                                  save_experiment=True)
                print("approach: {} - use_missing_indicator_variables: {} - n_features: {}".format(approach, feature_missing, n_features))
                exp.fit_predict(num_cv=16)
                display(exp.performances_df)


approach: ebm - use_missing_indicator_variables: Floating Bubbles - n_features: 10


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.866,0.469,0.911,0.23,0.98,0.516,0.285,0.9,0.446,0.535,0.938,0.469,0.951,0.465,0.531,0.049


approach: ebm - use_missing_indicator_variables: DogInGrassC - n_features: 10


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.855,0.46,0.901,0.225,0.978,0.511,0.289,0.898,0.44,0.535,0.935,0.46,0.951,0.465,0.54,0.049


approach: ebm - use_missing_indicator_variables: DogInGrassRRL - n_features: 10


...

,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.856,0.405,0.896,0.151,0.927,0.454,0.241,0.856,0.384,0.605,0.882,0.347,0.956,0.395,0.653,0.044


approach: ebm - use_missing_indicator_variables: No touches at all - n_features: 32


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.861,0.422,0.897,0.172,0.928,0.464,0.264,0.852,0.4,0.651,0.873,0.346,0.96,0.349,0.654,0.04


approach: ebm - use_missing_indicator_variables: Game grouped touches - n_features: 32


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.86,0.413,0.897,0.154,0.927,0.484,0.264,0.854,0.428,0.698,0.87,0.357,0.965,0.302,0.643,0.035


## Experiment 2  `scale_data`

In [46]:

for i, n_features in enumerate(list(features['xgboost'].keys())):
    for j, scenario in enumerate(['papers']):
        
        for scale_data in [True, False]:
            
            for approach in ["LogisticRegression", "DecisionTree", "nam", "ebm", 'xgboost']:
                
                try:
                    df =  pd.read_csv(AUTISM_DATA_PATH)


                    data = Dataset(df=df, 
                                   missing_data_handling='encoding',
                                   imputation_method='without',
                                   sampling_method='without',
                                   scenario = scenario, 
                                   features_name = features[approach][n_features] if approach in features.keys() else features['ebm'][n_features],
                                   scale_data=scale_data, 
                                   use_missing_indicator_variables=False,
                                   verbosity=1, 
                                   proportion_train=1)

                    exp = Experiments(data.dataset_name,
                                      dataset=data, 
                                      approach=approach, 
                                      previous_experiment=None,        
                                      debug=True, 
                                      verbosity=0, 
                                    experiment_folder_name='paper_experiment_2_fs',
                                      save_experiment=True)
                    exp.fit_predict(num_cv=16)

                    print("approach: {} - scale_data: {} - n_features: {}".format(approach, scale_data, n_features))
                    exp.fit_predict(num_cv=16)
                    display(exp.performances_df)                    

                except:
                    print("Faileed on ", scenario, n_features, approach)
                    pass

approach: LogisticRegression - scale_data: True - n_features: 10


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.73,0.264,0.796,0.082,0.851,0.37,0.181,0.834,0.283,0.488,0.87,0.28,0.943,0.512,0.72,0.057


approach: DecisionTree - scale_data: True - n_features: 10


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.659,0.372,0.878,0.202,0.964,0.406,0.319,0.915,0.359,0.256,0.983,0.611,0.927,0.744,0.389,0.073


approach: nam - scale_data: True - n_features: 10


...

,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV),Area Under the Curve (AUC)
0,0.7824,0.2815,0.7835,0.097,0.779,0.3622,0.1731,0.8118,0.29,0.5487,0.839,0.2777,0.9487,0.4513,0.7223,0.0513,0.7824


approach: ebm - scale_data: False - n_features: 32


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.859,0.415,0.906,0.159,0.929,0.475,0.259,0.863,0.408,0.628,0.887,0.365,0.958,0.372,0.635,0.042


approach: xgboost - scale_data: False - n_features: 32


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.852,0.472,0.929,0.242,0.982,0.494,0.351,0.9,0.422,0.488,0.942,0.467,0.947,0.512,0.533,0.053


## Experiment 3. `Imputation approach`

In [40]:

for i, n_features in enumerate(list(features['xgboost'].keys())):
    for j, scenario in enumerate(['papers']):
        
        for imputation_method in ['without','mice', 'knn', 'mean']:
            
            for approach in ["LogisticRegression", "DecisionTree", "nam", "ebm", 'xgboost']:
                
                try:
                    df =  pd.read_csv(AUTISM_DATA_PATH)

                    data = Dataset(df=df, 
                                   missing_data_handling='imputation',
                                   imputation_method=imputation_method,
                                   sampling_method='without',
                                   scenario = scenario, 
                                   features_name = features[approach][n_features] if approach in features.keys() else features['ebm'][n_features],
                                   scale_data=True, 
                                   use_missing_indicator_variables=False,
                                   verbosity=1, 
                                   proportion_train=1)

                    exp = Experiments(data.dataset_name,
                                      dataset=data, 
                                      approach=approach, 
                                      previous_experiment=None,        
                                      debug=True, 
                                     experiment_folder_name='paper_experiment_3_fs',
                                      verbosity=0, 
                                      save_experiment=True)

                    print("approach: {} - imputation_method: {} - n_features: {}".format(approach, imputation_method, n_features))
                    exp.fit_predict(num_cv=16)
                    display(exp.performances_df)                     

                except:
                    print("Failed on ", scenario, n_features, approach, imputation_method)
                    pass

approach: LogisticRegression - imputation_method: without - n_features: 10
Failed on  papers 10 LogisticRegression without
approach: DecisionTree - imputation_method: without - n_features: 10
Failed on  papers 10 DecisionTree without
approach: nam - imputation_method: without - n_features: 10
Failed on  papers 10 nam without
approach: ebm - imputation_method: without - n_features: 10
Failed on  papers 10 ebm without
approach: xgboost - imputation_method: without - n_features: 10


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.826,0.474,0.932,0.256,0.982,0.52,0.348,0.893,0.451,0.581,0.925,0.446,0.955,0.419,0.554,0.045


approach: LogisticRegression - imputation_method: mice - n_features: 10


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.732,0.259,0.801,0.079,0.795,0.367,0.182,0.863,0.277,0.395,0.911,0.315,0.936,0.605,0.685,0.064


approach: DecisionTree - imputation_method: mice - n_features: 10


...

,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.843,0.461,0.929,0.256,0.984,0.476,0.283,0.878,0.403,0.558,0.911,0.393,0.952,0.442,0.607,0.048


approach: LogisticRegression - imputation_method: mean - n_features: 32


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.829,0.493,0.928,0.267,0.984,0.5,0.374,0.924,0.462,0.372,0.981,0.667,0.938,0.628,0.333,0.062


Failed on  papers 32 DecisionTree mean
Failed on  papers 32 nam mean
Failed on  papers 32 ebm mean
approach: xgboost - imputation_method: mean - n_features: 32


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.839,0.471,0.924,0.251,0.983,0.455,0.335,0.867,0.38,0.558,0.899,0.364,0.952,0.442,0.636,0.048


## Experiment 4. `Sampling approach`

In [ ]:
n=0
for i, n_features in enumerate(list(features['xgboost'].keys())):
    for j, scenario in enumerate(['papers']):
        
        for sampling_method in ['without', 'vanilla', 'smote']:
            
            for approach in ["LogisticRegression", "DecisionTree", "nam", "ebm", 'xgboost']:
                    
                df =  pd.read_csv(AUTISM_DATA_PATH)


                data = Dataset(df=df, 
                               missing_data_handling='encoding',
                               imputation_method='without',
                               sampling_method=sampling_method,
                               scenario = scenario, 
                                   features_name = features[approach][n_features] if approach in features.keys() else features['ebm'][n_features],
                               scale_data=True, 
                               use_missing_indicator_variables=False,
                               verbosity=1, 
                               proportion_train=1)

                exp = Experiments(data.dataset_name,
                                  dataset=data, 
                                  approach=approach, 
                                  previous_experiment=None,        
                                  debug=True, 
                                 experiment_folder_name='paper_experiment_4_fs',
                                  verbosity=0, 
                                  save_experiment=True)
                print("approach: {} - sampling_method: {} - n_features: {}".format(approach, sampling_method, n_features))
                exp.fit_predict(num_cv=16)
                display(exp.performances_df)  
       

approach: LogisticRegression - sampling_method: vanilla - n_features: 2


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.768,0.377,0.875,0.153,0.923,0.441,0.238,0.854,0.367,0.581,0.882,0.338,0.953,0.419,0.662,0.047


approach: DecisionTree - sampling_method: vanilla - n_features: 2


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.799,0.281,0.746,0.084,0.633,0.453,0.2,0.871,0.363,0.512,0.909,0.367,0.947,0.488,0.633,0.053


approach: nam - sampling_method: vanilla - n_features: 2


...

approach: LogisticRegression - sampling_method: smote - n_features: 5


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.788,0.409,0.903,0.204,0.976,0.412,0.264,0.911,0.361,0.302,0.974,0.542,0.931,0.698,0.458,0.069


approach: DecisionTree - sampling_method: smote - n_features: 5


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.791,0.256,0.726,0.073,0.633,0.426,0.182,0.865,0.35,0.512,0.901,0.349,0.947,0.488,0.651,0.053


approach: nam - sampling_method: smote - n_features: 5


# Experiement 5 - Use of Clinical variables 

In [14]:
additional_clinical_variables = ['srs_total_tscore','srs_social_awareness_tscore','srs_social_motivation_tscore','cbcl_scaleIV_score','cbcl_asd_score','mchat_final']

In [15]:
n=0
for i, n_features in enumerate(list(features['ebm'].keys())):
    for use_clinical in [True, False]:

        df =  pd.read_csv(AUTISM_DATA_PATH)


        data = Dataset(df=df, 
                       missing_data_handling='encoding',
                       imputation_method='without',
                       sampling_method='without',
                       scenario = scenario, 
                       features_name = features['ebm'][n_features] + additional_clinical_variables if use_clinical  else features['ebm'][n_features],
                       scale_data=False, 
                       use_missing_indicator_variables=False,
                       verbosity=1, 
                       proportion_train=1)

        exp = Experiments(data.dataset_name,
                          dataset=data, 
                          approach='ebm', 
                          previous_experiment=None,        
                          debug=True, 
                         experiment_folder_name='paper_experiment_5_fs_papers',
                          verbosity=0, 
                          save_experiment=True)
        print("use_clinical: {} - n_features: {}".format(use_clinical, n_features))
        exp.fit_predict(num_cv='loocv')
        display(exp.performances_df)  


use_clinical: True - n_features: 10


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.94,0.855,0.992,0.732,0.998,0.786,0.69,0.959,0.749,0.744,0.981,0.8,0.974,0.256,0.2,0.026


use_clinical: False - n_features: 10


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.823,0.453,0.919,0.224,0.977,0.517,0.289,0.904,0.447,0.512,0.945,0.489,0.949,0.488,0.511,0.051


use_clinical: True - n_features: 22


...

,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.828,0.405,0.89,0.188,0.97,0.439,0.243,0.898,0.365,0.395,0.95,0.447,0.938,0.605,0.553,0.062


use_clinical: True - n_features: 32


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.945,0.855,0.993,0.724,0.998,0.818,0.664,0.963,0.784,0.814,0.978,0.795,0.981,0.186,0.205,0.019


use_clinical: False - n_features: 32


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.815,0.405,0.905,0.169,0.92,0.455,0.276,0.852,0.388,0.628,0.875,0.342,0.958,0.372,0.658,0.042


In [16]:
additional_clinical_variables = ['mchat_final']

In [17]:
n=0
for i, n_features in enumerate(list(features['ebm'].keys())):
    for use_clinical in [True, False]:

        df =  pd.read_csv(AUTISM_DATA_PATH)


        data = Dataset(df=df, 
                       missing_data_handling='encoding',
                       imputation_method='without',
                       sampling_method='without',
                       scenario = scenario, 
                       features_name = features['ebm'][n_features] + additional_clinical_variables if use_clinical  else features['ebm'][n_features],
                       scale_data=False, 
                       use_missing_indicator_variables=False,
                       verbosity=1, 
                       proportion_train=1)

        exp = Experiments(data.dataset_name,
                          dataset=data, 
                          approach='ebm', 
                          previous_experiment=None,        
                          debug=True, 
                         experiment_folder_name='paper_experiment_5_fs_papers',
                          verbosity=0, 
                          save_experiment=True)
        print("use_clinical: {} - n_features: {}".format(use_clinical, n_features))
        exp.fit_predict(num_cv='loocv')
        display(exp.performances_df)  


use_clinical: True - n_features: 10


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.913,0.752,0.986,0.564,0.997,0.725,0.55,0.95,0.685,0.651,0.981,0.778,0.965,0.349,0.222,0.035


use_clinical: False - n_features: 10


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.823,0.453,0.919,0.224,0.977,0.517,0.289,0.904,0.447,0.512,0.945,0.489,0.949,0.488,0.511,0.051


use_clinical: True - n_features: 22


...

,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.828,0.405,0.89,0.188,0.97,0.439,0.243,0.898,0.365,0.395,0.95,0.447,0.938,0.605,0.553,0.062


use_clinical: True - n_features: 32


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.917,0.706,0.971,0.442,0.984,0.727,0.57,0.946,0.683,0.721,0.969,0.705,0.971,0.279,0.295,0.029


use_clinical: False - n_features: 32


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.815,0.405,0.905,0.169,0.92,0.455,0.276,0.852,0.388,0.628,0.875,0.342,0.958,0.372,0.658,0.042


In [11]:
additional_clinical_variables = ['mchat_final']

In [12]:
n=0
for i, n_features in enumerate(list(features['ebm'].keys())):
    for use_clinical in [True, False]:

        df =  pd.read_csv(AUTISM_DATA_PATH)


        data = Dataset(df=df, 
                       missing_data_handling='encoding',
                       imputation_method='without',
                       sampling_method='without',
                       scenario = scenario, 
                       features_name = features['ebm'][n_features] + additional_clinical_variables if use_clinical  else features['ebm'][n_features],
                       scale_data=True, 
                       use_missing_indicator_variables=False,
                       verbosity=1, 
                       proportion_train=1)

        exp = Experiments(data.dataset_name,
                          dataset=data, 
                          approach='ebm', 
                          previous_experiment=None,        
                          debug=True, 
                         experiment_folder_name='paper_experiment_5_fs_papers',
                          verbosity=0, 
                          save_experiment=True)
        print("use_clinical: {} - n_features: {}".format(use_clinical, n_features))
        exp.fit()
        exp.dataset.proportion_train = 0.
        exp.dataset._init_scenario(scenario='papers_remote')
        exp.predict()

        display(exp.performances_df)
            

use_clinical: True - n_features: 10


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.835,0.751,0.728,0.114,0.598,0.763,0.25,0.76,0.535,0.837,0.702,0.679,0.851,0.163,0.321,0.149


use_clinical: False - n_features: 10


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.722,0.65,0.463,0.118,0.9,0.685,0.13,0.67,0.356,0.744,0.614,0.593,0.761,0.256,0.407,0.239


use_clinical: True - n_features: 22


...

,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.678,0.641,0.375,0.161,0.943,0.634,0.208,0.54,0.2,0.884,0.281,0.481,0.762,0.116,0.519,0.238


use_clinical: True - n_features: 32


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.833,0.793,0.687,0.306,0.985,0.757,0.377,0.74,0.519,0.884,0.632,0.644,0.878,0.116,0.356,0.122


use_clinical: False - n_features: 32


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.698,0.646,0.424,0.178,0.938,0.661,0.245,0.65,0.309,0.698,0.614,0.577,0.729,0.302,0.423,0.271
